#구글 드라이브 마운트 및 경로 설정

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

#필요 라이브러리 다운로드

youtube api를 사용하기 위한 google api 라이브러리를 설치한다.


In [ ]:
! pip install --upgrade google-api-python-client
! pip install oauth2client

In [ ]:
!pip install urllib

In [ ]:
import pandas as pd

In [ ]:
from IPython.display import Image
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import urllib
import cv2
import urllib.request
import requests

#이미지 저장 path 설정
뒤에 작업 후 이미지를 저장할 경로를 설정하거나 %cd 명령어를 통해 경로를 이동해두세요

In [ ]:
my_path = '/content/drive/MyDrive/goorm/goorm-project-3-machine_translation/data/youtube_img'

In [ ]:
!ls -a

#API Build 및 채널 search 함수 설정
DEVELOPER_KEY에는 유튜브 API 신청 후 받은 본인의 키를 스트링으로 넣어주시면 됩니다.

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY='본인 API 키'
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube=build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

가져올 동영상 목록의 개수를 바꾸고 싶으면 아래의 maxResults 값을 바꿔주시면 됩니다(default = 50개, API에서 지원하는 max 값도 50개 입니다.).

In [ ]:
def youtube_channel_search(channel_name):
    search_response=youtube.search().list(
        q=channel_name,
        order='relevance',
        part='snippet',
        maxResults=50,
        ).execute()
    return search_response

#img url을 반환해줄 함수 & img 저장 함수 선언

v_1 수정안 : save 과정에서 img파일의 url에 대한 status가 404인 경우(해당 url에 접근했을 때 이미지 파일이 없는 경우)간단한 에러 코드와 함께 저장하지 않도록 수정.

v_2 수정안 : 썸네일이 없는 영상이 아닌 maxresdefault 형식의 썸네일을 지원하지 않는 경우 hqdefault(가장기본값)으로 반환될 수 있도록 수정

In [ ]:
def make_img_url(videoId):
    img_url = "https://img.youtube.com/vi/" + videoId + "/maxresdefault.jpg"
    status = requests.get(img_url)
    if status.status_code == 404:
        print('maxres img를 찾을 수 없어 hqdefault로 반환합니다.')
        img_url = "https://img.youtube.com/vi/" + videoId + "/hqdefault.jpg"
        return img_url
    else:
        return img_url

In [ ]:
def save_imgfile(file_name, img_url):
    status = requests.get(img_url)
    if status.status_code == 404:
        print('해당 img를 찾을 수 없습니다 Error_code : 404')
        return
    else:
        with open(os.path.join(my_path, file_name), 'wb') as f:
            f.write(urllib.request.urlopen(img_url).read())
        return

#검색으로 동영상 ID 가져오기

해당 파트에서는 유튜브에 특정 단어 검색시 나오는 영상의 리스트를 가져와 리스트 안의 동영상들의 ID를 저장하여 img url을 반환하는 파트입니다.
search_key에 검색할 단어를 넣어주시면 됩니다

In [ ]:
search_key = '검색키'
search_list=youtube_channel_search(search_key)['items']

검색한 동영상들의 목록의 타이틀과 고유 ID를 확인할 수 있습니다.

In [ ]:
video_list=[]
for idx, video in enumerate(search_list) :
    try : 
        video_list.append([video['snippet']['title'], video['id']['videoId']])
    except : 
        print(f'{idx}번 째 목록의 video title 혹은 video ID가 존재하지 않습니다.')
        pass
video_list[:5]

썸네일을 가져올 땐 고유 동영상 ID 만 필요하므로 따로 저장해줍니다.

In [ ]:
videoid_list = [video[1] for video in video_list]

특정 동영상의 썸네일을 임시로 확인해볼 수 있습니다.
확인해볼 동영상의 index를 넣어주시면 됩니다.

In [ ]:
test = make_img_url(videoid_list[3])

test2 = "https://img.youtube.com/vi/" + videoid_list[0] + "/hqdefault.jpg"
Image(test2)


아래 for문을 실행하면 위에서 검색 당시 노출된 영상들의 썸네일이 저장경로에 저장됨을 확인할 수 있으며 파일 명은 검색어+idx로 저장되도록 설정해두었습니다.
간혹 썸네일이 없는 영상들의 경우 Not Found error가 발생해 try except로 넘겨버렸습니다.

In [ ]:
for idx, videoId in enumerate(videoid_list) :
    print(idx, f'{videoId}영상 저장 중 ...', end = ' ')
    img_url = make_img_url(videoId)
    file_name = search_key + '_'+str(idx)
    save_imgfile(file_name, img_url)
    print('영상 저장 완료.')
print('모든 작업이 끝났습니다.')


In [ ]:
def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)

In [ ]:
get_files_count('/content/drive/MyDrive/goorm/goorm-project-3-machine_translation/data/youtube_img')

#특정 채널의 재생목록 내의 동영상 목록 별로 ID 가져오기
해당 파트에서는 검색한 내용의 가장 위에 노출된 영상 채널의 재생목록에 접근해 원하는 재생 목록 내의 동영상들의 ID를 저장하여 img url을 반환하는 파트입니다.

원하는 검색어로 검색한 후 상단에 노출된 동영상 리스트를 반환 합니다. 리스트를 통해 본인이 원하는 채널이 있는지 확인하고 해당 영상의의 index를 아래 video_index에 넣어주시면 됩니다.(default = 0)

In [ ]:
search_items=youtube_channel_search('예시유튜브채널')['items']
search_items[:3]

In [ ]:
video_index = 0
channel_name = search_items[video_index]['snippet']['channelTitle']
channel_name

In [ ]:

channel_id=search_items[video_index]['snippet']['channelId']
channel_id

가져온 채널의 재생목록들을 반환합니다.

In [ ]:
playlists=youtube.playlists().list(
    channelId=channel_id,
    part='snippet',
    maxResults=20).execute()

In [ ]:
ids=[]
titles=[]
for i in playlists['items']:
    ids.append(i['id'])
    titles.append(i['snippet']['title'])
    
df=pd.DataFrame([ids,titles]).T
df.columns=['PlayLists','Titles']

재생목록들을 데이터 프레임으로 저장하여 목록들을 확인합니다.
재생목록중에서 본인이 원하는 재생목록의 index를 선택하여 playlist_index에 넣어줍니다.(default=0)

In [ ]:
print(df.head(20)) #playlist ID 값

In [ ]:
len(df)

20

In [ ]:
#영상 list
playlist_index = 4
dtcu=df['PlayLists'][playlist_index]
playlist_videos=youtube.playlistItems().list(
    playlistId=dtcu,
    part='snippet',
    maxResults=50)



해당 재생 목록 안의 동영상 목록들을 확인하고 dataframe으로 넣어줍니다.
**이 과정에서 해당 채널의 private 영상도 같이 포함이 되는데 private 영상의 경우 썸네일이나 정보를 가져올 수 없으니 꼭 확인하여 슬라이싱을 통해 private 영상은 제거하고 밑에 for문에 넣어주세요**

In [ ]:
playlistitems_list_response=playlist_videos.execute()

video_names=[]
video_ids=[]
date=[]

for v in playlistitems_list_response['items']:
    video_names.append(v['snippet']['title'])
    video_ids.append(v['snippet']['resourceId']['videoId'])
    date.append(v['snippet']['publishedAt'])
    
vdf=pd.DataFrame([date,video_names,video_ids]).T
vdf.columns=['Date','Title','IDS']
vdf

In [ ]:
vdf['IDS']

위에서 private 영상을 모두 지웠다면 아래 셀을 실행하면 따로 문제없이 파일들이 설정한 경로에 저장되있음을 확인하실 수 있습니다.

In [ ]:
for idx, videoId in enumerate(vdf['IDS']) :
    print(idx, videoId)
    img_url = make_img_url(videoId)
    file_name = channel_name + '_제목_'+str(idx)
    try:
        save_imgfile(file_name, img_url)
    except:
        pass